In [1]:
using LinearAlgebra
using SparseArrays
using Printf

In [2]:
import MathProgBase
MPB = MathProgBase

import Mosek:MosekSolver

In [3]:
solver = MosekSolver();

# TSSP formulation

Deterministic equivalent:

\begin{align*}
    (DEP) \ \ \ \min_{x, y} \ \ \ & c^{T}x + \sum_{i} p_{i} q_{i}^{T} y_{i}\\
    s.t. \ \ \ 
    & Ax = b,\\
    & T_{i} x + W_{i}y_{i} = h_{i}, \ \ \ \forall i,\\
    & x \geq 0,\\
    & y_{i} \geq 0, \ \ \ \forall i.
\end{align*}

Dual of (DEP) writes:
\begin{align*}
    (D) \ \ \ \max_{\eta, \theta} \ \ \ & b^{T}\eta + \sum_{i} h_{i}^{T} \theta_{i}\\
    s.t. \ \ \ 
    & A^{T} \eta + \sum_{i} T_{i}^{T} \theta_{i} \leq c,\\
    & W_{i}^{T} \theta_{i} \leq p_{i} q_{i}, \ \ \ \forall i.
\end{align*}

We wolve (D) by column generation. Note that it amounts to the same as solving (DEP) by Benders.

The Master problem writes
\begin{align*}
    (D) \ \ \ \min_{\eta, \theta} \ \ \ & -b^{T}\eta - \sum_{i, \omega} h_{i, \omega} \lambda_{i, \omega} - \sum_{i, \rho} h_{i, \rho} \lambda_{i, \rho}\\
    s.t. \ \ \ 
    & A^{T} \eta + \sum_{i, \omega} t_{i, \omega} \lambda_{i, \omega} + \sum_{i, \rho} t_{i, \rho} \lambda_{i, \rho} \leq c, & [\pi]\\
    & \sum_{i} \lambda_{i, \omega} = 1, \ \ \ \forall i, & [\sigma_{i}]\\
    & \lambda \geq 0.
\end{align*}

and the sub-problem writes:
\begin{align*}
    (SP) \ \ \ \min_{\theta_{i}} \ \ \ & (-h_{i}^{T} - \pi^{T} T_{i}^{T}) \theta_{i} - \sigma_{i}\\
    s.t. \ \ \ 
    & W_{i}^{T} \theta_{i} \leq p_{i} q_{i}.
\end{align*}

# Source code

In [4]:
include("src/TSSP/parser/core_parser.jl")
include("src/TSSP/parser/time_parser.jl")
include("src/TSSP/parser/stoch_parser.jl")

parse_block_line! (generic function with 1 method)

In [5]:
function generate_scenario_data(
        A, T, W, q, h,
        row2idx, col2idx, 
        prob, coeffs
)
    m1, n1 = size(A)
    m2, n2 = size(W)
    m2 == size(T, 1) || error("T and W has different number of rows.")
    n1 == size(T, 2) || error("T and A has different number of columns.")
    
    # Create problem data
    h_ = copy(h)          # Right-hand side
    q_ = prob .* copy(q)  # Objective ()
    
    T_ = copy(T)          # T_r matrix
    W_ = copy(W)          # W_r
    
    
    for (k, v) in coeffs
        row::Int = row2idx[k[1]]
        col::Int = col2idx[k[2]]
        
        row == 0 || row > m1 || error("Entry given for first-stage row $(k[1])")
        
        # Check four cases
        if row == 0 && col > 0
            # Objective coefficient
#             println("Obj: $k: $v")
            q_[col - n1] = v
            
        elseif row > 0 && col == 0
            # Right-hand side
#             println("Rhs: $k \t $v")
            h_[row - m1] = v
            
        elseif row > 0 && col > 0
            # Check whether belongs to T or W
            if col <= n1
                T[row - m1, col] = v
            else
                W[row - m1, col - n1] = v
            end
        else
            eror("Wrong combination of row/col indices: $k")
        end
    end
    
    return T_, W_, q_, h_
end

generate_scenario_data (generic function with 1 method)

# Test instance files

In [48]:
f_cor = "data/tssp/4node.cor"
f_sto = "data/tssp/4node.sto.1024"
f_tim = "data/tssp/4node.tim"

"data/tssp/4node.tim"

# Parse test instance

## Parse `.cor` file

In [49]:
# Extract data from .cor file
d, row2idx, col2idx, I, J, V, objI, objV, rhsI, rhsV, senses = parse_cor_file(f_cor);

## Parse `.tim` file

In [50]:
name, is_lp, col_periods, row_periods = parse_time_file(f_tim);

In [51]:
# Extract indices of first/second period rows/columns
col1, col2 = (
    col2idx[col_periods[1]]:(col2idx[col_periods[2]]-1),  # First-period variables
    col2idx[col_periods[2]]:d["ncols"]  # Second-period variables
)
row1, row2 = (
    row2idx[row_periods[1]]:(row2idx[row_periods[2]]-1),  # First-period rows
    row2idx[row_periods[2]]:d["nrows"]  # Second-period variables
)

(1:14, 15:88)

In [52]:
# Extract vector of objective coefficients
obj = sparsevec(objI, objV, d["ncols"])
c, q = Vector(obj[col1]), Vector(obj[col2])

([50.0, 65.0, 45.0, 70.0, 60.0, 55.0, 50.0, 60.0, 55.0, 40.0  …  76.8, 57.6, 62.4, 67.2, 57.6, 38.4, 43.2, 57.6, 43.2, 62.4], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [53]:
# Extract right-hand side vectors
rhs = sparsevec(rhsI, rhsV, d["nrows"])
b, h = Vector(rhs[row1]), Vector(rhs[row2])

([25.0, 25.0, 25.0, 25.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 480.0, 240.0, 0.0, 0.0], [5.0, 8.0, 4.0, 4.5, 6.7, 4.2, 10.1, 8.3, 12.2, 3.2  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [54]:
# Extract constraint matrices
A_ = sparse(I, J, V);

A = A_[row1, col1]
T = A_[row2, col1]
W = A_[row2, col2];

## Parse `.sto` file

In [55]:
name, indeps, blocks = parse_sto_file(f_sto);

In [56]:
# Extract each scenario from .sto data
S_ = Base.Iterators.product(values(indeps)..., values(blocks)...)
S = [prod(collect(s)) for s in S_]
length(S)

1024

# Construct deterministic equivalent problem

In [57]:
pb_data = [generate_scenario_data(A, T, W, q, h, row2idx, col2idx, s.p, s.values) for s in S][:];

In [58]:
T_ = [x[1] for x in pb_data];
W_ = [x[2] for x in pb_data];
q_ = [x[3] for x in pb_data];
h_ = [x[4] for x in pb_data];

In [59]:
m1, n1 = size(A)
m2, n2 = size(W)
R = length(S)

1024

In [60]:
@time U = [
    hcat(
        T_[r],
        spzeros(m2, n2*(r-1)),
        W_[r],
        spzeros(m2, n2*(R-r))
    )
    for r in 1:R
];

@time vU = vcat(U...);

  4.071030 seconds (115.68 k allocations: 4.372 GiB, 13.49% gc time)
 19.749797 seconds (389.43 M allocations: 5.818 GiB, 3.25% gc time)


In [61]:
rhs_ = vcat(b, h_...)
obj_ = vcat(c, q_...)
senses_ = vcat(senses[1:m1], repeat(senses[(m1+1):end], R))
n_ = length(obj_)
m_ = length(rhs_)
A_ = vcat(
    hcat(A,spzeros(m1, n2*R)),
    vU
);

rhs_l = rhs_ .- Inf .* (senses_ .== "L");
rhs_u = rhs_ .+ Inf .* (senses_ .== "G");

In [65]:
size(A_), nnz(A_)

((75790, 190516), 479520)

In [69]:
airl = MPB.LinearQuadraticModel(solver)
MPB.loadproblem!(airl, A_, zeros(n_), Inf*ones(n_), obj_, rhs_l, rhs_u, :Min);

MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (1) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (2) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (4) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (5) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (7) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (8) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (10) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (12) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (14) of matrix 'A'.
MOSEK warning 710: #1 (nearly) zero elements are specified in sparse col '' (15) of matrix 'A'.
Warning number 710 is disabled.


In [70]:
MPB.optimize!(airl)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 75790           
  Cones                  : 0               
  Scalar variables       : 190516          
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 4096
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.02            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.26    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimizati

In [71]:
MPB.getobjval(airl)

434.11250000000007

# Column-generation formulation

In [72]:
import Linda

## Create oracles

In [74]:
pool = Linda.Oracle.LindaOraclePool()

MethodError: MethodError: no method matching Linda.Oracle.LindaOraclePool()
Closest candidates are:
  Linda.Oracle.LindaOraclePool(!Matched::Any) at /mnt/c/Users/mathi/GitHub/Linda.jl/src/Oracle/oracle_pool.jl:23

In [ ]:
rowlb = q_[1] .- Inf .* (senses_ .== "L");
rowub = q_[1] .+ Inf .* (senses_ .== "G");
Linda.Oracle.LindaOracleMIP(
    1,
    h_[1], T_[1]', W_[1]',
    rowlb, rowub,
    
)

In [95]:
size(A')

(52, 14)